In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from google.colab import drive
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
!pip install keras-tuner
from tensorflow.keras import mixed_precision
import keras_tuner as kt
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)
keras.utils.set_random_seed(9)
drive.mount('/content/drive')

     |████████████████████████████████| 133 kB 4.3 MB/s 
INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: Tesla T4, compute capability 7.5
Mounted at /content/drive


In [ ]:
injury_history = pd.read_csv('/content/drive/MyDrive/data_490/injury_data/injury_history')
bio_data = pd.read_csv('/content/drive/MyDrive/data_490/processed_data/bio_data.csv')
ts_data = pd.read_pickle('/content/drive/MyDrive/data_490/processed_data/thirty_day_timestep_df')

In [ ]:
bio_data.player_age = bio_data['player_age'].str.split(' ').str[0]

In [ ]:
indexes = np.random.permutation(len(ts_data))
train_size = int(len(indexes)*.8)
valid_size = int(train_size*.2)
train_indexes = indexes[valid_size:train_size]
valid_indexes = indexes[:valid_size]
test_indexes = indexes[train_size:]

In [ ]:
bio_data = bio_data.fillna(-1)
injury_history = injury_history.fillna(-1)

In [ ]:
ts_train = np.stack(ts_data.loc[train_indexes, 'time_series'].values).astype(np.float16)[:,:,:-1]
ts_valid = np.stack(ts_data.loc[valid_indexes, 'time_series'].values).astype(np.float16)[:,:,:-1]
ts_test = np.stack(ts_data.loc[test_indexes, 'time_series'].values).astype(np.float16)[:,:,:-1]
bio_train = bio_data.iloc[train_indexes, 2:].astype(np.float16)
bio_valid = bio_data.iloc[valid_indexes, 2:].astype(np.float16)
bio_test = bio_data.iloc[test_indexes, 2:].astype(np.float16)
injury_train = injury_history.iloc[train_indexes, 2:].astype(np.float16)
injury_valid = injury_history.iloc[valid_indexes, 2:].astype(np.float16)
injury_test = injury_history.iloc[test_indexes, 2:].astype(np.float16)
train_target =  ts_data.loc[train_indexes, 'injured'].astype(np.float16)
valid_target =  ts_data.loc[valid_indexes, 'injured'].astype(np.float16)
test_target = ts_data.loc[test_indexes, 'injured'].astype(np.float16)

In [ ]:
class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, dropout=0, time_steps=30, features=85):
        super().__init__()
        self.dropout = layers.Dropout(dropout)
        p = np.zeros(np.shape((1, time_steps, features)))
        columns = np.arange(features)
        rows = np.arange(time_steps)
        p = rows.reshape(-1,1)/1000**(columns*2/features)
        p[:, 0::2] = np.sin(p[:,0::2])
        p[:, 1::2] = np.cos(p[:, 1::2])
        self.p = p

    def call(self, x, **kwargs):
        x = x + self.p
        return self.dropout(x)

In [ ]:
from re import X
def transformer_encoder(inputs, head_size, num_heads, conv_filters, kernel_size, dropout=0):
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=conv_filters, kernel_size=kernel_size, activation="relu", padding='same')(x)
    X = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

In [ ]:
def build_mixed_model(
    ts_input_shape,
    injury_input_shape,
    bio_input_shape,
    head_size=256,
    num_heads=8,
    conv_filters=64,
    kernel_size=3,
    num_transformer_blocks=4,
    mlp_units=128,
    mlp_layers=2,
    encoder_dropout=0,
    mlp_dropout=0,
    pe_dropout=0,
    embedding_dropout=0,
    pe=True,
    embedding=True,
):
    ts_input = keras.Input(shape=ts_input_shape)
    x = ts_input
    x = keras.layers.Normalization(axis=1)(x)
    
    #Embedding
    if embedding:
      x = layers.Flatten(input_shape=ts_input_shape)(x)
      x = layers.Dense(ts_input_shape[-1]*ts_input_shape[-2], activation='tanh')(x)
      x = layers.Dropout(embedding_dropout)(x)
      x = layers.Reshape(ts_input_shape)(x)

    #Positional Encoding
    if pe:
      x = PositionalEncoding()(x)
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, conv_filters, kernel_size, encoder_dropout)


    bio_input = keras.Input(shape=bio_input_shape)
    bio = layers.experimental.preprocessing.Normalization()(bio_input)

    injury_input = keras.Input(shape=injury_input_shape) 
    injury = layers.experimental.preprocessing.Normalization()(injury_input)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    x = tf.keras.layers.Concatenate()([x, injury, bio])

    for i in range(mlp_layers):
        x = layers.Dense(mlp_units, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    inputs = [ts_input, injury_input, bio_input]
    outputs = layers.Dense(1, activation="sigmoid")(x)
    return keras.Model(inputs, outputs)

In [ ]:
[2**i for i in range(5,9)]

[32, 64, 128, 256]

In [ ]:
def model_search(hp):
  
  ts_shape = np.shape(ts_train)[1:]
  injury_shape = np.shape(injury_train)[1:]
  bio_shape = np.shape(bio_train)[1:]

  model = build_mixed_model(
  ts_shape,
  injury_shape,
  bio_shape,
  head_size = hp.Choice('head_size' , [2**i for i in range(5,10)]),
  num_heads = hp.Choice('num_heads', [2**i for i in range(2,6)]),
  conv_filters = hp.Choice('conv_filters', [2**i for i in range(4,8)]),
  num_transformer_blocks = hp.Int('num_encoder_blocks', 2, 5),
  mlp_units=hp.Choice('mlp_units', [2**i for i in range(5,9)]),
  mlp_layers=hp.Int('mlp_layers', 1, 4),
  encoder_dropout = hp.Float('encoder_dropout', 0, .5),
  mlp_dropout = hp.Float('mlp_dropout', 0, .5),
  pe_dropout = hp.Float('pe_dropout', 0, .5),
  embedding_dropout= hp.Float('embedding_dropout', 0, .5),
  pe= hp.Boolean('pe', default=True),
  embedding= hp.Boolean('embedding', default=True)
  )

  opt = keras.optimizers.Adam(learning_rate=hp.Float('lr', 1e-7, 1e-3))
  
  
  model.compile(
    loss=keras.losses.BinaryCrossentropy(),
    optimizer=opt,
    metrics=[keras.metrics.AUC()],
    steps_per_execution=32,
    jit_compile=True
  )
  return model

In [ ]:
tuner = kt.RandomSearch(
    model_search,
    'val_loss',
    max_trials=100)

In [ ]:
callbacks = [keras.callbacks.EarlyStopping(patience=10)]
tuner.search([ts_train, injury_train, bio_train],
            train_target,
           validation_data=[[ts_valid, injury_valid, bio_valid], valid_target],
           epochs=100, callbacks=callbacks,
           batch_size=256)

Trial 65 Complete [00h 05m 35s]
val_loss: 0.08804738521575928

Best val_loss So Far: 0.08229991048574448
Total elapsed time: 19h 56m 24s

Search: Running Trial #66

Value             |Best Value So Far |Hyperparameter
64                |128               |head_size
8                 |32                |num_heads
32                |32                |conv_filters
7                 |4                 |num_encoder_blocks
128               |128               |mlp_units
4                 |3                 |mlp_layers
0.015989          |0.28705           |encoder_dropout
0.052893          |0.14918           |mlp_dropout
0.14986           |0.25219           |pe_dropout
0.48895           |0.077008          |embedding_dropout
False             |True              |pe
False             |False             |embedding
0.00093291        |0.00091911        |lr

Epoch 1/100
331/331 [==============================] - 92s 277ms/step - loss: 25.5259 - auc: 0.4911 - val_loss: 0.2083 - val_auc: 0.5787


In [1]:
print(tuner.get_best_hyperparameters(num_trials=1)[0])

NameError: ignored